In [7]:
import os
from PIL import Image, UnidentifiedImageError
from fpdf import FPDF
import traceback

def image_to_pdf(image_path, output_pdf_path):
    """
    긴 이미지를 A4 크기로 잘라서 PDF를 생성하는 함수.

    매개변수:
        image_path (str): 입력 이미지 파일 경로.
        output_pdf_path (str): 생성할 PDF 파일 경로.
    """
    try:
        # 1) 이미지 불러오기
        try:
            image = Image.open(image_path)
            image.load()  # 실제 이미지 로드
            print(f"[DEBUG] 이미지 불러오기 성공: {image_path}")
        except FileNotFoundError:
            print(f"[ERROR] 파일을 찾을 수 없습니다: {image_path}")
            return
        except UnidentifiedImageError:
            print(f"[ERROR] 이미지 파일이 아니거나 손상된 파일입니다: {image_path}")
            return
        except Exception as e:
            print(f"[ERROR] 이미지 로드 중 예외 발생: {e}")
            traceback.print_exc()
            return

        # 2) A4 사이즈 설정 (pt 단위)
        a4_width_pt = 595
        a4_height_pt = 842

        # 3) pt → px 변환 (72 DPI 가정)
        dpi = 72
        a4_width_px = int(a4_width_pt * dpi / 72)
        a4_height_px = int(a4_height_pt * dpi / 72)

        # 4) 이미지가 A4보다 크면 상단 기준으로 자르기
        if image.width > a4_width_px or image.height > a4_height_px:
            left = 0
            top = 0
            right = min(a4_width_px, image.width)
            bottom = min(a4_height_px, image.height)
            image = image.crop((left, top, right, bottom))
            print(f"[DEBUG] 이미지 crop 수행: ({left}, {top}, {right}, {bottom})")
        else:
            print("[DEBUG] 이미지가 A4 이하 크기이므로 crop 생략")

        # 5) 임시 이미지 저장
        temp_image_path = "resized_image.png"
        try:
            image.save(temp_image_path)
            print(f"[DEBUG] 임시 이미지 저장: {temp_image_path}")
        except Exception as e:
            print(f"[ERROR] 이미지 저장 중 예외 발생: {e}")
            traceback.print_exc()
            return

        # 6) PDF 생성 및 이미지 삽입
        try:
            pdf = FPDF(unit="pt", format="A4")
            pdf.add_page()
            pdf.image(temp_image_path, x=0, y=0, w=a4_width_pt)
        except Exception as e:
            print(f"[ERROR] PDF 생성 중 예외 발생: {e}")
            traceback.print_exc()
            return

        # 7) PDF 저장
        try:
            pdf.output(output_pdf_path)
            
            # 여기에서 '파란색' 출력 + 경로를 파일:// 형태로 감싸면
            # 일부 터미널/IDE(예: iTerm2, 일부 VSCode 터미널)에서는 클릭 가능한 링크가 될 수 있음
            link_path = f"file://{os.path.abspath(output_pdf_path)}" 
            # \033[94m = 파란색 텍스트 시작, \033[0m = 컬러 리셋
            print(f"\033[94m[PDF 생성 완료] → {link_path}\033[0m")
        except Exception as e:
            print(f"[ERROR] PDF 저장 중 예외 발생: {e}")
            traceback.print_exc()
            return
    finally:
        # 8) 임시 파일 정리
        if os.path.exists("resized_image.png"):
            try:
                os.remove("resized_image.png")
                print("[DEBUG] 임시 이미지 파일 삭제 완료")
            except Exception as e:
                print(f"[WARNING] 임시 파일 삭제 중 예외 발생: {e}")
                traceback.print_exc()


# 사용 예시
if __name__ == "__main__":
    test_image_path = "/Users/Juna/Downloads/carbon (2).png"
    test_output_pdf_path = "output.pdf"
    image_to_pdf(test_image_path, test_output_pdf_path)


[DEBUG] 이미지 불러오기 성공: /Users/Juna/Downloads/carbon (2).png
[DEBUG] 이미지 crop 수행: (0, 0, 595, 842)
[DEBUG] 임시 이미지 저장: resized_image.png
[PDF 생성 완료] → file:///Volumes/Juna/touslesjuna/Made/output.pdf
[DEBUG] 임시 이미지 파일 삭제 완료
